기하학 : 점, 선, 면, 도형 등의 기하학적인 대상을 다루는 학문

기하학 처리 : 기하학적인 대상의 공간적 배치를 변경하는 과정

### 8.1 사상

사상 : 입력 영상의 좌표가 새롭게 배치될 목적 영상의 좌표를 찾아서 화소값을 옮기는 과정
- 순방향 사상 : 입력 영상의 좌표를 중심으로 목적 영상의 좌표를 계산 
  - 입력 영상과 목적 영상의 크기가 같을 때 사용함 -> 영상의 크기가 달라지면 홀이나 오버랩의 문제가 발생 <br><br>

- 역방향 사상 : 목적 영상의 좌표를 중심으로 역변환을 계산하여 원본 영상의 좌표를 찾아서 화소값을 가져옴
  - 홀이나 오버랩이 발생되지 않음
  - 입력 영상의 한 화소를 목적 영상의 여러 화소에서 사용하면 결과 영상의 품질이 떨어질 수 있음


홀 : 목적 영상의 좌표를 만드는 과정에서 사상되지 않은 화소

오버랩 : 여러 화소들이 목적 영상의 한 화소로 사상되는 현상

### 8.2 크기변경 (확대/축소)

In [4]:
# 8.2.1 영상 크기 변경
import numpy as np, cv2
import time

# 정방행렬 인덱스로 크기 변경
def scaling(img, size):
    dst = np.zeros(size[::-1], img.dtype)
    ratioY, ratioX = np.divide(size[::-1], img.shape[:2]) # 비율 계산
    y = np.arange(0, img.shape[0], 1)
    x = np.arange(0, img.shape[1], 1)
    y, x = np.meshgrid(y, x)
    i, j = np.int32(y * ratioY), np.int32(x * ratioX) # 목적 영상 좌표
    dst[i, j] = img[y, x]
    return dst

# 반복문을 이용하여 크기 변경
def scaling2(img, size):
    dst = np.zeros(size[::-1], img.dtype)
    ratioY, ratioX = np.divide(size[::-1], img.shape[:2]) # 비율 계산
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            i, j = int(y * ratioY), int(x * ratioX) # 목적 영상 좌표
            dst[i, j] = img[y, x]
    return dst

def time_check(func, image, size, title):
    start_time = time.perf_counter()
    ret_img = func(image, size)
    elapsed = (time.perf_counter() - start_time) * 1000
    print(title, "수행시간 = %0.2f ms" % elapsed)
    return ret_img


image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

dst1 = scaling(image, (150, 200))
dst2 = scaling2(image, (150, 200))
dst3 = time_check(scaling, image, (300, 400), "좌표 행렬 방식 :")
dst4 = time_check(scaling2, image, (300, 400), "반복문 방식 :")

cv2.imshow("image", image)
cv2.imshow("dst1", dst1)
cv2.imshow("dst3", dst3)
cv2.waitKey(0)



좌표 행렬 방식 : 수행시간 = 27.03 ms
반복문 방식 : 수행시간 = 807.74 ms


-1

### 8.3 보간

##### 8.3.1 최근접 이웃 보간법
최근접 이웃 보간법 : 목적 영상을 만드는 과정에서 홀이 된 위치는 가장 가깝게 이웃한 입력 영상의 화소값을 가져옴
- 쉽고 빠르게 목적 영상의 품질을 높일 수 있음
- 경계선이나 모서리 부분에서 계단 현상이 나타날 수 있음

In [6]:
# 8.3.1 크기변경 (최근접 이웃 보간법)
import numpy as np, cv2
from Common.interpolation import scaling

def scaling_nearest(img, size):
    dst = np.zeros(size[::-1], img.dtype)
    ratioY, ratioX = np.divide(size[::-1], img.shape[:2]) # 비율 계산
    i = np.arange(0, size[1], 1)
    j = np.arange(0, size[0], 1)
    i, j = np.meshgrid(i, j)
    y, x = np.int32(i / ratioY), np.int32(j / ratioX)
    dst[i, j] = img[y, x]

    return dst

image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

dst1 = scaling(image, (350, 400))
dst2 = scaling_nearest(image, (350, 400))

cv2.imshow("image", image)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)
cv2.waitKey(0)

-1

##### 8.3.2 양선형 보간법

선형 보간을 두 번에 걸쳐서 수행함

In [4]:
# 8.3.1 크기변경 (양선형 보간법)
import numpy as np, cv2
from Common.interpolation import scaling_nearest

def bilinear_value(img, pt):
    x, y = np.int32(pt)
    if x >= img.shape[1]-1: x = x - 1
    if y >= img.shape[0]-1: y = y - 1

    P1, P2, P3, P4 = np.float32(img[y:y+2, x:x+2].flatten())
    
    alpha, beta = pt[1] - y, pt[0] - x
    M1 = P1 + alpha * (P3 - P1) # 1차 보간
    M2 = P2 + alpha * (P4 - P2) 
    P = M1 + beta * (M2 - M1) # 2차 보간
    return np.clip(P, 0, 255)

def scaling_bilinear(img, size):
    ratioY, ratioX = np.divide(size[::-1], img.shape[:2]) # 비율 계산

    dst = [[ bilinear_value(img, (j/ratioX, i/ratioY))
            for j in range(size[0])]
        for i in range(size[1])]
    return np.array(dst, img.dtype)

image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

size = (350, 400)
dst1 = scaling_bilinear(image, size)
dst3 = cv2.resize(image, size, 0, 0, cv2.INTER_LINEAR)
dst4 = cv2.resize(image, size, 0, 0, cv2.INTER_NEAREST)

cv2.imshow("image", image)
cv2.imshow("user_bilinear", dst1)
cv2.imshow("cv_bilinear", dst3)
cv2.imshow("cv_Nearest", dst4)
cv2.waitKey(0)

-1